In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
import pyxdf
import pandas as pd
import re

from jupyter_client import channels

In [23]:
filename = 'data\\P104\\sub-P104_ses-Omnideck_S002_task-Default_run-001_eeg.xdf'
streams, fileheader = pyxdf.load_xdf(filename, synchronize_clocks=False)
match = re.sub(r"_ses-", "-", filename.split("_task-Default_run")[0])
eeg_stream = next(s for s in streams if "EEG" in s['info']['type'][0])
eeg_data = np.array(eeg_stream['time_series']).T

eeg_timestamps = np.array(eeg_stream['time_stamps'])
sfreq = float(eeg_stream['info']['nominal_srate'][0])

ch_names = []
for ch in eeg_stream['info']['desc'][0]['channels'][0]['channel']:
    ch_names.append(ch['label'][0])

info = mne.create_info(
    ch_names=ch_names[:-3],
    sfreq=sfreq,
    ch_types='eeg'
)

raw = mne.io.RawArray(eeg_data[:-3]/10e5, info)

montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)

marker_streams = [stream for stream in streams if stream['info']['name'][0] == 'LSL4Unity.OmnideckWaiterVR']

found likely XDF file corruption (unclosed token: line 1, column 3047), ignoring corrupted XML element in footer.
Stream 3: Calculated effective sampling rate 3870.9120 Hz is different from specified rate 100.0000 Hz.


Creating RawArray with float64 data, n_channels=32, n_times=253538
    Range : 0 ... 253537 =      0.000 ...   507.074 secs
Ready.


In [22]:
streams[0]['info']

defaultdict(list,
            {'name': ['ViconMarkerData'],
             'type': ['MotionCapture'],
             'channel_count': ['5'],
             'channel_format': ['float32'],
             'source_id': ['vicon12345'],
             'nominal_srate': ['100.0000000000000'],
             'version': ['1.100000000000000'],
             'created_at': ['76187.47533360000'],
             'uid': ['781a623f-8108-488c-948f-2a9d4c7fcb2e'],
             'session_id': ['default'],
             'hostname': ['OMNIDECK'],
             'v4address': [None],
             'v4data_port': ['16572'],
             'v4service_port': ['16574'],
             'v6address': [None],
             'v6data_port': ['16572'],
             'v6service_port': ['16574'],
             'desc': [None],
             'stream_id': 2,
             'effective_srate': np.float64(3884.965683535985)})

In [24]:
# Adjust for whatever markers are of interest
nr_1008 = 0
nr_1009 = 0
nr_1010 = 0
for i in marker_streams:
    for marker in i['time_series']:
        if marker == ['1008']:
            nr_1008 += 1
        if marker == ['1009']:
            nr_1009 += 1
        if marker == ['1010']:
            nr_1010 += 1
    print(i['time_series'])
    # To compare timing of the markers
    #print(i['time_stamps'])
print('Marker 1008:',nr_1008)
print('Marker 1009:',nr_1009)
print('Marker 1010:',nr_1010)
print(len(marker_streams))

[['1001'], ['1002'], ['1004'], ['1018'], ['1023'], ['1024'], ['1025'], ['1026'], ['1012'], ['1012'], ['1010'], ['1027'], ['1026'], ['1012'], ['1010'], ['1011'], ['1019'], ['1016'], ['1014'], ['1027'], ['1028'], ['1029'], ['1022'], ['1023'], ['1024'], ['1025'], ['1026'], ['1012'], ['1012'], ['1012'], ['1012'], ['1012'], ['1010'], ['1027'], ['1026'], ['1012'], ['1012'], ['1027'], ['1026'], ['1012'], ['1012'], ['1010'], ['1011'], ['1019'], ['1016'], ['1014'], ['1027'], ['1028'], ['1012'], ['1012'], ['1012'], ['1029'], ['1022'], ['1023'], ['1024'], ['1025'], ['1026'], ['1012'], ['1010'], ['1027'], ['1026'], ['1012'], ['1010'], ['1011'], ['1019'], ['1016'], ['1014'], ['1027'], ['1028'], ['1029'], ['1022'], ['1012'], ['1023'], ['1024'], ['1025'], ['1026'], ['1012'], ['1012'], ['1010'], ['1012'], ['1012'], ['1027'], ['1026'], ['1012'], ['1010'], ['1005'], ['1011'], ['1019'], ['1016'], ['1017'], ['1027'], ['1028'], ['1029'], ['1022'], ['1006'], ['1007'], ['1034'], ['1035'], ['1001'], ['1015'],

In [63]:
np.array(marker_streams[0]['time_series'][1:]) == np.array(marker_streams[1]['time_series'])
# 19 Tables, 2 Bottlers per table

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [

## The following code loads all the files, takes the longest marker stream counts the occurences of each marker in that stream. Additionally it loops through all other streams and counts how often 1008 and 1009 are present and saves this into a txt file


In [2]:
# This can be used to run it for all participants, I put the files for each participant in individual folders with no subfolders
subjects = ['P115'] #'P101', 'P104'
path = 'data\\'
frontal_ch = ['F3', 'Fz', 'F4','FC1', 'FC2', 'C3', 'Cz', 'C4', 'CP1' ,'CP2']
for id_ in subjects:
    for filename in sorted(os.listdir(path+id_)):
        if filename == 'sub-P001_ses-Omnideck_S001_task-Default_run-001_eeg.xdf':
            continue
        streams, fileheader = pyxdf.load_xdf(path+id_+'/'+filename, synchronize_clocks=False)
        match = re.sub(r"_ses-", "-", filename.split("_task-Default_run")[0])
        #print(match)
        #eeg_stream = next(s for s in streams if "EEG" in s['info']['type'][0])
        #eeg_data = np.array(eeg_stream['time_series']).T

        #eeg_timestamps = np.array(eeg_stream['time_stamps'])
        #sfreq = float(eeg_stream['info']['nominal_srate'][0])

        #ch_names = []
        #for ch in eeg_stream['info']['desc'][0]['channels'][0]['channel']:
        #    ch_names.append(ch['label'][0])

        #info = mne.create_info(
            #ch_names=ch_names[:-3],
            #sfreq=sfreq,
            #ch_types='eeg'
        #)

        #raw = mne.io.RawArray(eeg_data[:-3]/10e5, info)

        #montage = mne.channels.make_standard_montage('standard_1020')
        #raw.set_montage(montage)
        marker_streams = [stream for stream in streams if stream['info']['name'][0] == 'LSL4Unity.OmnideckWaiterVR']
        # Initialize combined lists for time_series and time_stamps
        mStream_count = len(marker_streams)
        longest_mStream = []
        count_1008 = 0
        count_1009 = 0
        for s in marker_streams:
            if len(s['time_series']) >= len(longest_mStream):
                longest_mStream = s
            for entry in s['time_series']:
                if entry == ['1008']:
                    count_1008 += 1
                if entry == ['1009']:
                    count_1009 += 1


        marker_stream = {
            'info': marker_streams[0]['info'],
            'time_series': np.array(longest_mStream['time_series']),
            'time_stamps': np.array(longest_mStream['time_stamps']),
        }
        with open(f'markers\\{match}_markers.txt', 'w') as file:
            i = 0
            my_bool = True
            overall_count = 0
            file.write(f'Total number of Marker streams {len(marker_streams)}\n')
            while my_bool:
                #if i == 8:
                #    file.write(f'{1000+i} is present {count_1008} times \n')
                #elif i == 9:
                #    file.write(f'{1000+i} is present {count_1009} times \n')
                #else:
                file.write(f'{1000+i} is present {np.sum(marker_stream['time_series']== str(1000+i))} times \n')
                overall_count += np.sum(marker_stream['time_series']==str(1000+i))
                i += 1
                if i ==38:
                    my_bool = False
            file.write(f'Overall {overall_count} Markers are found in the file')

Stream 1: Calculated effective sampling rate 1988.2226 Hz is different from specified rate 100.0000 Hz.
Stream 1: Calculated effective sampling rate 2385.4943 Hz is different from specified rate 100.0000 Hz.
Stream 1: Calculated effective sampling rate 3370.4983 Hz is different from specified rate 100.0000 Hz.
Stream 1: Calculated effective sampling rate 3282.9716 Hz is different from specified rate 100.0000 Hz.
Stream 1: Calculated effective sampling rate 3445.9816 Hz is different from specified rate 100.0000 Hz.
Stream 2: Calculated effective sampling rate 3318.9372 Hz is different from specified rate 100.0000 Hz.
Stream 2: Calculated effective sampling rate 3856.9995 Hz is different from specified rate 100.0000 Hz.
Stream 3: Calculated effective sampling rate 3860.8859 Hz is different from specified rate 100.0000 Hz.


In [43]:
np.sum(marker_stream['time_series'] == '1002')

np.int64(3)